## 问题定义

问题定义如下图，矩形求解区域，长宽为0.1m，上侧面为从左至右的1m/s的滑移边界，其他三个侧面为无滑移边界。雷诺数为10。
![Problem](./resource/problem_define.png)

对待求解问题的数学描述如下：
![Equation](./resource/equation.png)

其中，$u,v$分别为x方向速度，y方向速度，$p$为压强，$\nu,\rho$分别为运动粘度和密度。在本求解中定义$\nu=0.01$，$\rho=1.0$

### 求解目标

给定坐标$(x,y)$求解稳态结果（$u,v, p$）

## Importance Sampling

本例将引入重要性采样，从而实现加速


## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
# 必要的符号运算
from sympy import Symbol, Eq, Abs

import modulus.sym

# 超参数
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.hydra.utils import compose

# 求解器
from modulus.sym.solver import Solver

# domain
from modulus.sym.domain import Domain

# 几何物体
from modulus.sym.geometry.primitives_2d import Rectangle

# 约束
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)

# validator
from modulus.sym.domain.validator import PointwiseValidator

# inferencer
from modulus.sym.domain.inferencer import PointwiseInferencer
from modulus.sym.key import Key

# Equation
from modulus.sym.eq.pdes.navier_stokes import NavierStokes

# 构建IS的必要组件
from modulus.sym.graph import Graph
from modulus.sym.domain.constraint import Constraint

# post process
from modulus.sym.utils.io import (
    csv_to_dict,
    ValidatorPlotter,
    InferencerPlotter,
)
import matplotlib.pyplot as plt

In [3]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 10000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: 1000
  rec_inference_freq: 2000
  rec_monitor_freq: 1000
  rec_constraint_freq: 2000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: true
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arch:
  fully_connected:
    arch_type: fully_connected
    input_keys: ???
  

### 定义必要组件

#### PDE

In [4]:
ns = NavierStokes(nu=0.01, rho=1.0, dim=2, time=False)

#### Model

In [5]:

# 定义简单的全连接网络
# 输入为坐标x和y
# 输出为两个方向的速度u,v以及压强p
flow_net = instantiate_arch(
        input_keys=[Key("x"), Key("y")],
        output_keys=[Key("u"), Key("v"), Key("p")],
        cfg=cfg.arch.fully_connected,
    )
print(flow_net)

# 创建node集合
nodes = ns.make_nodes() + [flow_net.make_node(name="flow_network")]

FullyConnectedArch(
  (_impl): FullyConnectedArchCore(
    (layers): ModuleList(
      (0): FCLayer(
        (linear): WeightNormLinear(in_features=2, out_features=512, bias=True)
      )
      (1-5): 5 x FCLayer(
        (linear): WeightNormLinear(in_features=512, out_features=512, bias=True)
      )
    )
    (final_layer): FCLayer(
      (activation_fn): Identity()
      (linear): Linear(in_features=512, out_features=3, bias=True)
    )
  )
)


#### Importance Sampling Graph

在Modulus中使用IS需要定义对应的计算图和度量方法

In [6]:
# 定义计算图
# 用于计算进行重要性度量的必要参数
importance_model_graph = Graph(
        nodes,
        invar=[Key("x"), Key("y")],
        req_names=[
            Key("u", derivatives=[Key("x")]),
            Key("u", derivatives=[Key("y")]),
            Key("v", derivatives=[Key("x")]),
            Key("v", derivatives=[Key("y")]),
        ],
    ).to('cuda')

In [7]:
# 定义重要性度量方法
def importance_measure(invar):
        outvar = importance_model_graph(
            Constraint._set_device(invar, device='cuda', requires_grad=True)
        )
        importance = (
            outvar["u__x"] ** 2
            + outvar["u__y"] ** 2
            + outvar["v__x"] ** 2
            + outvar["v__y"] ** 2
        ) ** 0.5 + 10
        return importance.cpu().detach().numpy()

#### Geo

In [8]:
# 长宽均为1的矩形
height = 0.1
width = 0.1
x, y = Symbol("x"), Symbol("y")
rec = Rectangle((-width / 2, -height / 2), (width / 2, height / 2))

#### Domain

在Domain中定义约束以及训练所需的各种组件

In [9]:
# make ldc domain
ldc_domain = Domain()

#### 边界条件

四个边界条件

In [10]:
# 顶部滑移边界
# 水平速度为1.0，垂直速度为0
# 注意lambda_weighting参数，这个参数定义了样本权重，在本示例中，约靠近边界，样本的权重越小
# criteria定义了采样的位置
# outvar定义了边界条件
top_wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": 1.0, "v": 0},
    batch_size=cfg.batch_size.TopWall,
    lambda_weighting={"u": 1.0 - 20 * Abs(x), "v": 1.0},  # weight edges to be zero
    criteria=Eq(y, height / 2),
    importance_measure=importance_measure,  # 引入重要性度量模块
)
ldc_domain.add_constraint(top_wall, "top_wall")

# 左右以及下侧的无滑移边界
no_slip = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": 0, "v": 0},
    batch_size=cfg.batch_size.NoSlip,
    criteria=y < height / 2,
    importance_measure=importance_measure,  # 引入重要性度量模块
)
ldc_domain.add_constraint(no_slip, "no_slip")

#### PDE约束

内部满足PDE约束

In [11]:
# interior
# 由于本求解中NS方程直接调用的模型，所以这里outvar直接使用了对应的key
# 具体定义可参考modulus/eq/pdes/navier_stokes.py
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"continuity": 0, "momentum_x": 0, "momentum_y": 0},
    batch_size=cfg.batch_size.Interior,
    lambda_weighting={
        "continuity": Symbol("sdf"),
        "momentum_x": Symbol("sdf"),
        "momentum_y": Symbol("sdf"),
    },
    importance_measure=importance_measure,  # 引入重要性度量模块
)
ldc_domain.add_constraint(interior, "interior")

#### Validator & Inferencer

验证器以及其他必要组件

In [12]:
# 数据目录
file_path = "openfoam/cavity_uniformVel0.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {"Points:0": "x", "Points:1": "y", "U:0": "u", "U:1": "v", "p": "p"}
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_var["x"] += -width / 2  # center OpenFoam data
    openfoam_var["y"] += -height / 2  # center OpenFoam data
    
    # 输入数据
    openfoam_invar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    # 输出数据
    openfoam_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["u", "v"]
    }
    
    # 创建validator
    openfoam_validator = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        true_outvar=openfoam_outvar_numpy,
        batch_size=1024,
        plotter=ValidatorPlotter(),
    )
    ldc_domain.add_validator(openfoam_validator)

    # 创建inferencer
    grid_inference = PointwiseInferencer(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        output_names=["u", "v", "p"],
        batch_size=1024,
        plotter=InferencerPlotter(),
    )
    ldc_domain.add_inferencer(grid_inference, "inf_data")
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

### 求解器以及求解

In [13]:
# 定义求解器
slv = Solver(cfg, ldc_domain)

手动加载日志系统

In [14]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [15]:
slv.solve()

2024-02-27 08:37:46,738 - modulus.sym.trainer - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-02-27 08:37:46,748 - modulus.sym.trainer - INFO - attempting to restore from: /workspace/09_2D_LDC_Importance_Sampling/outputs
2024-02-27 08:37:46,749 - modulus.sym.trainer - WARNING - optimizer checkpoint not found
2024-02-27 08:37:46,750 - modulus.sym.trainer - WARNING - model flow_network.0.pth not found
2024-02-27 08:38:12,814 - modulus.sym.trainer - INFO - [step:          0] record constraint batch time:  1.370e-01s
2024-02-27 08:38:21,207 - modulus.sym.trainer - INFO - [step:          0] record validators time:  8.392e+00s
2024-02-27 08:38:26,427 - modulus.sym.trainer - INFO - [step:          0] record inferencers time:  5.217e+00s
2024-02-27 08:38:26,461 - modulus.sym.trainer - INFO - [step:          0] saved checkpoint to /workspace/09_2D_LDC_Importance_Sampling/outputs
2024-02-27 08:38

2024-02-27 08:45:32,703 - modulus.sym.trainer - INFO - [step:       4700] loss:  2.244e-04, time/iteration:  6.950e+01 ms
2024-02-27 08:45:39,776 - modulus.sym.trainer - INFO - [step:       4800] loss:  1.948e-04, time/iteration:  7.071e+01 ms
2024-02-27 08:45:46,597 - modulus.sym.trainer - INFO - [step:       4900] loss:  1.623e-04, time/iteration:  6.819e+01 ms
2024-02-27 08:46:30,365 - modulus.sym.trainer - INFO - [step:       5000] record validators time:  7.681e+00s
2024-02-27 08:46:30,396 - modulus.sym.trainer - INFO - [step:       5000] saved checkpoint to /workspace/09_2D_LDC_Importance_Sampling/outputs
2024-02-27 08:46:30,398 - modulus.sym.trainer - INFO - [step:       5000] loss:  2.061e-04, time/iteration:  4.380e+02 ms
2024-02-27 08:46:37,714 - modulus.sym.trainer - INFO - [step:       5100] loss:  1.562e-04, time/iteration:  7.315e+01 ms
2024-02-27 08:46:44,271 - modulus.sym.trainer - INFO - [step:       5200] loss:  1.749e-04, time/iteration:  6.554e+01 ms
2024-02-27 08:4

2024-02-27 08:55:20,592 - modulus.sym.trainer - INFO - [step:      10000] record validators time:  7.587e+00s
2024-02-27 08:55:25,275 - modulus.sym.trainer - INFO - [step:      10000] record inferencers time:  4.680e+00s
2024-02-27 08:55:25,306 - modulus.sym.trainer - INFO - [step:      10000] saved checkpoint to /workspace/09_2D_LDC_Importance_Sampling/outputs
2024-02-27 08:55:25,307 - modulus.sym.trainer - INFO - [step:      10000] loss:  1.251e-04, time/iteration:  5.271e+02 ms
2024-02-27 08:55:25,308 - modulus.sym.trainer - INFO - [step:      10000] reached maximum training steps, finished training!


### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

如果使用了PointwiseValidator则可以直接查看验证的结果：

![u](./outputs/validators/validator_u.png)
![v](./outputs/validators/validator_v.png)